# Module 8: GIS and mapping

**In this module, we will learn the basics of working with spatial data and conducting spatial network analysis.**

More specifically, we will introduce the following tools:

1. Basic spatial data and GIS operations using [geopandas](https://geopandas.org/), which spatializes pandas dataframes. It uses the [shapely](https://shapely.readthedocs.io/en/latest/manual.html) package for geometry.
2. Spatial network modeling and analysis with [OSMnx](https://osmnx.readthedocs.io), which allows you to download, model, analyze, and visualize street networks (and other spatial data) anywhere in the world from [OpenStreetMap](https://www.openstreetmap.org/#map=4/38.01/-95.84), an open source geodatabase that provides freely accessible spatial data worldwide.


This module has two major sections:  
1. [Working with spatial data with geopandas](#spatialdata)
2. [Spatial network](#spatialnetwork)

Relevant readings:  
- Gimond. 2021. Intro to GIS and Spatial Analysis, ch. 1, 2, 9. [Direct link](https://mgimond.github.io/Spatial/introGIS.html).
- Boeing and Arribas-Bel. 2021. GIS and Computational Notebooks. In: The Geographic Information Science & Technology Body of Knowledge, edited by J.P. Wilson. [Direct link](https://gistbok.ucgis.org/bok-topics/gis-and-computational-notebooks)
- Kontokosta. 2018. Urban Informatics in the Science and Practice of Planning. Journal of Planning Education and Research. [Direct link](https://journals.sagepub.com/doi/10.1177/0739456X18793716).
- O'Sullivan. 2014. Spatial Network Analysis. Handbook of Regional Science, edited by Fischer and Nijkamp. [Direct link](https://link.springer.com/referenceworkentry/10.1007%2F978-3-642-23430-9_67)



In [ ]:
import pandas as pd
import geopandas as gpd
import osmnx as ox

%matplotlib inline

<a id='spatialdata'></a>
# 1. Working with spatial data

## 1.1 What is GIS?

GIS stands for geographic information system, a multi-component environment used to create, manage, visualize and analyze data and its spatial counterpart. Points, Lines and Polygons are three basic spatial elements of GIS vector data. GIS software lets you work with spatial data, that is, data associated with locations on the Earth. These locations are represented with coordinates: longitude (x), latitude (y), and often elevation (z). With GIS software you can collect, edit, query, analyze, and visualize spatial data. Examples of GIS software include ArcGIS, QGIS, and GeoPandas. 



## 1.2 Spatial data and GeoPandas

You can use a GIS software like ArcGIS or QGIS to open a spatial data file (typically a shapefile, GeoJSON file, or CSV file with lat-long columns). Today we'll introduce the basic concepts of spatial data and GIS operations using geopandas, an open source project to make working with geospatial data in Python. We'll focus on common, shared concepts and operations.

### 1.2a Loading a shapefile

A shapefile is a folder containing multiple files with spatial geometry, attribute data, projection information, etc: https://en.wikipedia.org/wiki/Shapefile

Where to get census shapefiles? https://www.census.gov/cgi-bin/geo/shapefiles/index.php



In [ ]:
# tell geopandas to read a shapefile with its read_file() function, passing in the shapefile folder
# this produces a GeoDataFrame
# the data I am using here contain all US county polygon
gdf = gpd.read_file('../data/cb_2013_us_county_5m/cb_2013_us_county_5m.shp')
gdf.shape

In [ ]:
# just like regular pandas, see the first 5 rows of the GeoDataFrame
# this is a shapefile of polygon geometries, that is, county boundaries
gdf.head()

In [ ]:
# mapping is as easy as calling the GeoDataFrame's plot method
ax = gdf.plot()

In [ ]:
# just like in regular pandas, we can filter and subset the GeoDataFrame
# exclude counties in island states
island = gdf['STATEFP'].isin(['15','69','66','02','72','60', '78'])
gdf_county = gdf[~island]

gdf_county.shape

In [ ]:
# what is the CRS?
# this derives from the shapefile's .prj file
# always make sure the shapefile you load has prj info so you get a CRS attribute!
gdf_county.crs

When we loaded the shapefile, geopandas loaded the CRS from the shapefile itself. But if you are loading a CSV file with lat lon geometry that is not explicitly spatial and it contains no CRS data, remember that you should always define the CRS manually.  

You need to look up an appropriate map projection (https://en.wikipedia.org/wiki/Map_projection) for the spatial extents of your data/map. This is a huge topic in and of itself, so for today we'll just focus on some (over-simplified) rules of thumb:

  1. If you're mapping global data, choose a global projection like [Mercator](https://spatialreference.org/ref/sr-org/16/) or [Robinson](https://spatialreference.org/ref/esri/54030/)
  1. If you're mapping national data, choose a national projection like [epsg:2163](https://spatialreference.org/ref/epsg/2163/) for the US

https://spatialreference.org/ is a good resource. There you can click the "proj4" link on any CRS's page to get a string you can use with geopandas.

Here in this case, I project the my county data to 'epsg:2163', a CRS appropriate for projecting USA data:

In [ ]:
# define a CRS appropriate for projecting USA data
usa_crs = 'epsg:2163'

gdf_county_proj = gdf_county.to_crs(usa_crs)

**Be careful**: heed the difference between `gdf.crs` and `gdf.to_crs()`. The first tells you the geodataframe's current CRS. The latter projects the geodataframe to a new CRS.

In [ ]:
# plot the projected county
ax = gdf_county_proj.plot()

### 1.2b Loading a CSV file

Here, we load a csv file with county-level demographic data from ACS, we want to merge this with the county shapefile in order to map those data

In [ ]:
# load acs county-level data as a regular pandas dataframe
# this is not spatial data as it has no geometry defined in the dataframe
df = pd.read_csv('../data/us_county_acs_data.csv')
df.shape

In [ ]:
df.head()

In [ ]:
df.columns

### 1.2c. Merge data (using pandas (non-spatial) merge)

In [ ]:
# now we have two dataset, one is the county level shapefile, the other is the csv file containing county-level demographic data
# we can perform merge using pandas based on their common field (county code) to combine the two dataset
# make sure the same data type of the common field
gdf_county_proj['GEOID'] = gdf_county_proj['GEOID'].astype(int)
gdf_county_proj = gdf_county_proj.set_index('GEOID').sort_values(by='GEOID')

df['fips'] = df['fips'].astype(int)
df = df.set_index('fips').sort_values(by='fips')

df_data = pd.merge(left=df, right=gdf_county_proj, how='inner', left_index=True, right_index=True)

In [ ]:
# create a new geopandas geodataframe manually from the pandas dataframe
gdf_data = gpd.GeoDataFrame(df_data)
gdf_data.columns

Now we have the geodataframe with county polygon geometry and the demographic data we want to map

## 1.3 Mapping

In [ ]:
# map using GeoDataFrame plot method with some style configurations
ax = gdf_data.plot(column='Total Population',
                        cmap='YlOrBr', edgecolor='k', lw=0.2, scheme='quantiles',
                        figsize=(9,6), legend=True)

# turn the "axis" off and save to disk
ax.axis('off')
ax.get_figure().savefig('pop.png', dpi=300, bbox_inches='tight')

In [ ]:
#alternatively, you could use matplotlib for more plotting features
import matplotlib.pyplot as plt

fig, ax = plt.subplots(figsize=(10, 10))
ax = gdf_data.plot(ax=ax, column='inclu_index', scheme='NaturalBreaks', k=6, cmap='YlOrBr', edgecolor='white', linewidth=0.5, legend=True, legend_kwds={'loc': 'lower left'})
ax.set_title('Racial Inclusion Index', fontsize=15)

ax.set_axis_off()

#fig.savefig('inclusion_index.png', dpi=600)
plt.show()

##### Racial inclusion  
| variables | measure | definition |  
|---- | --- | --- |  
| perc_minor | Percentage of color | Percentage of population made up by people of color |  
| raceedu_gap | Racial education gap | Racial disparities in education |
| race_homeownergap | Homeownership gap | Racial disparities in access to capital and wealth building opportunities |
| race_povertygap | Poverty gap | Racial disparities in economic depravation | 

Disclaimer: These indicators have not been fact-checked, they are what I calculated based on the method published by [Urban Institute](https://apps.urban.org/features/inclusion/?topic=map) for illutrating the mapping features in this module 

In [ ]:
cols = ['perc_minor', 'raceedu_gap', 'race_homeowngap', 'race_povertygap']

fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(20, 15))

for ax, col in zip(axes.flatten(), cols): 
    ax = gdf_data.plot(ax=ax, column=col, scheme='quantiles', k=6, cmap='YlOrBr', 
                       edgecolor='white', linewidth=0.5, legend=True, legend_kwds={'loc': 'lower left'})
    ax.set_title(col, fontsize=15)
    ax.set_axis_off()

# add a title to the figure
fig.suptitle('Figure 1: Spatial distribution of race inclusion indicators \n by U.S. Counties, 2015', y=0.95, fontsize=20, weight='bold')
fig.text(0.1, 0, 'Note: Scheme=quantiles; Unit=US county; Source=Census Bureau', fontsize=18)

#fig.savefig('race_inclusion.png', dpi=200)

plt.show()

**What stories can you tell from the above maps**

### Choosing color
https://matplotlib.org/users/colormaps.html

Easy rules to (usually) pick a good color map:

- if you have data values rising from some baseline to some maximum, use a perceptually uniform sequential color map.
- if you have data values diverging in both directions from some meaningful center point (e.g., center is zero and values can range positive or negative) then use a diverging color map
- avoid rainbow/jet color maps

### Geometric operations
https://geopandas.org/docs/user_guide/geometric_manipulations.html

GIS and spatial analysis use common "computational geometry" operations, such as intersects, within, and dissolve:

  - *intersects* tells you if each geometry in one dataset intersects with some other (single) geometry
  - *within* tells you if each geometry in one dataset is within some other (single) geometry
  - *dissolve* lets you aggregate data (merge their geometries together) if they share some attribute in common: this is the spatial equivalent of pandas's groupby function
  
If you want to learn more, check out [Geopandas example gallery](https://geopandas.org/gallery/index.html)

The Census API recently published [censusviz package](https://pypi.org/project/censusviz/) to help map Census data in Python via Census Population Estimate API and the Census Cartographic GeoJSON boundary files

<a id='spatialnetwork'></a>
# 2. Spatial network analysis and OSMnx

A network is a set of objects (called nodes or vertices) connected to each other by a set of connections (called edges or links). A graph is a mathematical model of a network: usually used synonymously. You can represent a graph as an adjacency matrix to use the tools of linear algebra to analyze it. 

A spatial network is a network that is spatially embedded. That means its nodes and/or edges have locations in space. A spatial network is defined by both its geometry (positions, distances, angles, etc) and its topology (connections and configurations). 

Examples:

  - street networks
  - airline routes
  - rail lines

We introduce a spatial network analysis tool OSMnx, which is built on top of GeoPandas, NetworkX, and matplotlib and interacts with OpenStreetMap’s APIs.

More info:

  - [OSMnx documentation](https://osmnx.readthedocs.io)
  - [Examples, demos, tutorials](https://github.com/gboeing/osmnx-examples)

## 2.1 Download and plot street network

In [ ]:
# model the walkable network within our original study site with OSMnx
place = 'Brookline, Massachusetts, USA'

# or make query an unambiguous dict to help the geocoder find specifically what you're looking for
place = {'city' : 'Brookline',
         'state' : 'Massachusetts',
         'country' : 'USA'}

G_walk = ox.graph_from_place(place, network_type='walk')
fig, ax = ox.plot_graph(G_walk)

OSMnx geocodes the query "Brookline, Massachusetts, USA" to retrieve the place boundaries of that city from the Nominatim API, retrieves the walkable street network data within those boundaries from the Overpass API, constructs a graph model, then simplifies/corrects its topology such that nodes represent intersections and dead-ends and edges represent the street segments linking them.   


In [ ]:
# you can get networks anywhere in the world
G = ox.graph_from_place('Vic, Spain', network_type='all')
fig, ax = ox.plot_graph(G, node_size=0, edge_linewidth=0.5)

In [ ]:
# or get network by address, coordinates, bounding box, or any custom polygon
# ...useful when OSM just doesn't already have a polygon for the place you want
NEU = (42.339949425548006, -71.08910732918775)  #search in the google map for the lat lon
one_mile = 1609 #meters
G = ox.graph_from_point(NEU, dist=one_mile, network_type='drive')
fig, ax = ox.plot_graph(G, node_size=0)

You can use this feature to get spatial street network basically anywhere around the world from OpenStreetMap. Try a place you know in your hometown (but note that for large urban areas, it could take hours to load the network!).

Other examples of getting networks by coordinates, bounding box, or any custom polygon shape: https://github.com/gboeing/osmnx-examples/blob/master/notebooks/01-overview-osmnx.ipynb

In [ ]:
# let's get back to the walkable network for the study site
type(G_walk)

OSMnx models all networks as NetworkX `MultiDiGraph` objects. You can convert to:

  - undirected NetworkX MultiGraphs
  - NetworkX DiGraphs without (possible) parallel edges
  - GeoPandas node/edge GeoDataFrames

In [ ]:
# convert your graph to node and edge GeoPandas GeoDataFrames
# extract node/edge GeoDataFrames, retaining only necessary columns 
nodes = ox.graph_to_gdfs(G_walk, edges=False)[['x', 'y']]
edges = ox.graph_to_gdfs(G_walk, nodes=False).reset_index()[['u', 'v']]

In [ ]:
nodes.head()

In [ ]:
edges.head()

You can create a graph from node/edge GeoDataFrames, as long as gdf_nodes is indexed by osmid and gdf_edges is multi-indexed by u, v, key (following normal MultiDiGraph structure). This allows you to load graph node/edge shapefiles or GeoPackage layers as GeoDataFrames then convert to a MultiDiGraph for graph analytics.

## 2.2 Get any geospatial entities' geometries and attributes

Use the `geometries` module to download entities, such as local amenities, points of interest, or building footprints, and turn them into a GeoDataFrame.  


In [ ]:
# get all building footprints in some neighborhood
place = 'Fenway park, Boston, Massachusetts'

tags = {'building': True}
gdf = ox.geometries_from_place(place, tags)

fig, ax = ox.plot_footprints(gdf, figsize=(3, 3))

In [ ]:
# get all parks and bus stops in some neighborhood
place = 'Boston, Massachusetts, USA'

tags = {'leisure': 'park',
        'highway': 'bus_stop'}

gdf = ox.geometries_from_place(place, tags)
gdf.shape

For more information about OSM taginfo and key value pairs, see https://wiki.openstreetmap.org/wiki/Tags#Keys_and_values and https://taginfo.openstreetmap.org/

In [ ]:
# restaurants near the school
address = '360 Huntington Ave, Boston, MA 02115'
tags = {'amenity' : 'restaurant'}
gdf = ox.geometries_from_address(address, tags=tags, dist=500)
gdf[['name', 'cuisine', 'geometry']].dropna().head()

In [ ]:
gdf.plot()

For more information about OSM taginfo and key value pairs, see https://wiki.openstreetmap.org/wiki/Tags#Keys_and_values and https://taginfo.openstreetmap.org/

In [ ]:
import matplotlib.pyplot as plt 

#add basemap
import contextily as ctx

def add_basemap(ax, zoom, source='http://tile.stamen.com/terrain/tileZ/tileX/tileY.png'):
    xmin, xmax, ymin, ymax = ax.axis()
    basemap, extent = ctx.bounds2img(xmin, ymin, xmax, ymax, zoom=zoom, source=source)
    ax.imshow(basemap, extent=extent, interpolation='bilinear')
    # restore original x/y limits
    ax.axis((xmin, xmax, ymin, ymax))
    
# define the project to work with the basemap
gdf_proj = gdf.to_crs(epsg=3857)

fig, ax = plt.subplots(figsize=(5, 5))
ax = gdf_proj.plot(ax=ax, marker='+', markersize=60, alpha=0.9)
add_basemap(ax, zoom=15, source=ctx.sources.ST_TONER_LITE)
ax.set_axis_off()

## 2.3 Routing - shortest path analysis

In [ ]:
# impute missing edge speeds then calculate edge (free-flow) travel times
G = ox.add_edge_speeds(G)
G = ox.add_edge_travel_times(G)

In [ ]:
# get the nearest network nodes to two lat/lng points
snell_library = (42.338, -71.088)
MFA = (42.339, -71.093)

# get the nearest network nodes to two lat/lng points with the distance module
orig = ox.distance.nearest_nodes(G, X=-71.088, Y=42.338)
dest = ox.distance.nearest_nodes(G, X=-71.093, Y=42.339)

In [ ]:
# find the shortest path between nodes, minimizing travel time, then plot it
route = ox.shortest_path(G, orig, dest, weight="travel_time")
fig, ax = ox.plot_graph_route(G, route, node_size=0)

In [ ]:
# how long is our route in meters?
edge_lengths = ox.utils_graph.get_route_edge_attributes(G, route, "length")
round(sum(edge_lengths))

You can do many things with OSMnx, please take a look at https://github.com/gboeing/osmnx-examples/blob/main/notebooks/00-osmnx-features-demo.ipynb for other features and more [example notebooks](https://github.com/gboeing/osmnx-examples)  

Take a look at this interesting publication for spatial analysis using OSMnx, written using the notebook:  
Boeing, G. (2020) [Urban Street Network Analysis in a Computational Notebook](https://openjournals.wu.ac.at/ojs/index.php/region/article/view/278), REGION, 6(3), pp. 39-51. doi: 10.18335/region.v6i3.278.

## This week's assignment - Assignment5

In this assignment, you will get your hands on spatial data. Follow the instructions available Canvas module for this week. 

Consider using dataset(s) and doing the analysis that would benefit your final project (but it is not required).
